In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
from gensim.models import Word2Vec
import spacy; from spacy.lang.en import English; nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
from utils import *
#import tensorflow_hub as hub

In [2]:
data = getData()

/home/pranav_pro/SchoolSystem/models/essay/utils.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data['illegible']=data.essay.str.contains('(\?\?\?|illegible|not legible)')


In [4]:
print(data.essay[0])

Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy

In [5]:
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def build_vocab(sentences, verbose =  True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [7]:
def getAllSentences(replaceNer, removeSW):
    sentences = []
    for essay in tqdm(data['essay'], disable=False):
        essay = basicClean(essay)
        essay = replaceNER(essay)
        essay = nlp(essay)
        for sent in essay.sents:
#             sent = sent.text
            sent = [token.text for token in sent if (not token.is_punct)]
            #print(type(sent))
#             sent = clean(sent,replaceNer=replaceNer, removeSW=removeSW)
#             ini = sent
#             sent = sent.split()
            #sent = [x.split() for x in sent]
            #essay = list(filter(None, essay))
            sentences.append(sent)
    return sentences

In [8]:
allSentences = getAllSentences(True,True)

100%|██████████| 12701/12701 [00:20<00:00, 631.29it/s]


In [10]:
allSentences[5]

['well',
 'now',
 'there',
 "'s",
 'a',
 'new',
 'way',
 'to',
 'chat',
 'the',
 'computer',
 'theirs',
 'plenty',
 'of',
 'sites',
 'on',
 'the',
 'internet',
 'to',
 'do',
 'so',
 '@organization',
 '@organization',
 '@caps',
 'facebook',
 'myspace',
 'ect']

In [11]:
vocab = build_vocab(allSentences)
import operator
sorted_x = sorted(vocab.items(), key=operator.itemgetter(1))[::-1]
sorted_x

100%|██████████| 155246/155246 [00:00<00:00, 232795.13it/s]


[('the', 149039),
 ('to', 97154),
 ('and', 78076),
 ('a', 60846),
 ('of', 52916),
 ('that', 51089),
 ('i', 48891),
 ('in', 43231),
 ('is', 40238),
 ('it', 38666),
 ('@caps', 34334),
 ('you', 32025),
 ('was', 30800),
 ('not', 30535),
 ('they', 25145),
 ('be', 24909),
 ('for', 24402),
 ('on', 23961),
 ('are', 21922),
 ('have', 20901),
 ('people', 20777),
 ('with', 19135),
 ('my', 18020),
 ('we', 17963),
 ('this', 17231),
 ('or', 16086),
 ('do', 15506),
 ('he', 15403),
 ('would', 15057),
 ('can', 14955),
 ('because', 14334),
 ('there', 13951),
 ('so', 13672),
 ('if', 13648),
 ('but', 12964),
 ('all', 12611),
 ('when', 12452),
 ('about', 11985),
 ('she', 11162),
 ('as', 11144),
 ('at', 10781),
 ('computers', 10680),
 ('had', 10446),
 ('what', 10364),
 ('like', 10039),
 ('their', 9882),
 ('from', 9766),
 ('one', 9586),
 ('computer', 9378),
 ('should', 9351),
 ('time', 9292),
 ('were', 9236),
 ('just', 9111),
 ('out', 8995),
 ('get', 8852),
 ('her', 8839),
 ('his', 8811),
 ('them', 8765),
 (

In [12]:
n=0
for w in vocab.keys():
    if vocab[w]<=3:
        n+=1
    if vocab[w]==1:
        print(w)
print(n)

globe(astronomy
perpressured
forbidde
mysace
erazer
garrenteed
stopp
descion
spme
studdies
catalouges
lam
coustmers
provid
commucated
catalouge
castomer
coustomers
jobless
convinences
conveint
convience
parttners
subssiquently
camputer
sitdown
wouldbe
ardthritis
revolutinizing
occures
intune
submerge
mealting
electricty
thjen
shorline
fluded
sientest
technolagy
peroson
goung
raed
cuzin
califan
exirsis
exmple
consiter
detaileds
incomputer
airs
ganes
exersus
evrye
ols
zap
interection
nessessity
usod
neurologist
scaresly
oviously
bility
technologicall
sectored
privalges
pleanty
hobbys
centainly
joggin
workin
aegre
tnachnolage
tnachnolige
halp
kohar
froi
alnsas
tnanchnolage
wod
tnachnologe
evans
maech
tnacholege
frchr
famas
invanyor
tnacholage
leki
lena
orde
dvanse
mashine
tnachologe
neccessitys
lonelyest
recestion
foregin
tye
tact
amp
barain
imprdove
refferances
vris
inapropreit
wepons
nickeloden.com
socally
excirsise
webkinz
invatations
wallpaper
grater
unseatled
friesnd
typee
understand

finatics
mysapce
edecation
saavy
motivator
inteligent
casheer
trademark
yousupposed
wwwe
comes.i
woops
corporal
chrildrens
soccser
anyining
maens
connectionds
sype
limts
talk/
enveironment
familieis
gagits
offtine
exerce
legue
proformances
aplication
erason
addicition
pedafile
advertizment
workplaces
dabt
encounterned
assuring
anda
excercizing
coordontion
posite
persades
reaqquainted
financialy
jeoporady
hospitials
radioshack
comuitateing
rackemed
saven
outsiad
wnight
coumite
shion
cowth
vite
abeast
computers.and
fullist
blivie
jackup
sisintsits
percentages
gofly
kites
downlosd
publicher
purposs
alloews
direclly
cetered
jumpstart
instan
sire
sharpen
advertion
abuce
botton
drivin
bace
strogger
professial
slappy
phonebill
grately
tastly
vidoe
hasal
canceless
switche
hesasity
scramble
listioning
scat
quizs
kuchers
hasin
confeti
questin
theyare
nano
invend
andd
bungy
jumpimg
tayst
grulling
chanels
excresing
plack
billed
sted
rtimes
insire
havethin
acroosthe
deatails
fittest
facebook/@caps


execis
tricking
encylipea
encyclipea
liks
easiness
downsized
scociety
today.if
pragraphs
comperters
strss
malking
harold
advancences
studdy
airconditioning
beang
necies
nephues
buuyed
quicly
decition
comonly
welcoms
photosynthesis
expierement
paparatzi
stified
dogged
dayrace
begger
fatler
fatts
gaim
moths
obesy
mortage
comiting
climer
murded
girths
carbon
dioxide
obesitey
inperson
playing.to
inside.for
neggative
defind
begetting
calarise
oldes
diebites
cholesteal
skillz
exploreing
maek
opinnion
grued
prettier
littyle
simulate
solatire
sereno
we'@caps
wioth
fastter
assignmnet
cuple
selp
informatrion
ip
pries
burdens
enabled
mysape
pedifiles
creepos
pursude
ajustment
soccerball
developer
reaso
exercise.well
esang.com
posonal
pasons
originallity
sports.
www.@caps
merrily
aglow
specialists
bussneses
corprat
missles
satlelite
plaines
nitro
atrack
classifieded
requirded
economist
becuause
exersce
flipout
hur
immence
spouses
marages
astoundingly
impubesent
nonesence
famillies
youv
contrivert

wacked
lovey
dovey
fun.also
aisl
cosomos
pleasureable
librabiens
cridical
children.a
posssible
phenomial
sacks
underrated
differentiate
another.i
cencord
it.becaus
it.books
offensivly
forthe
racesim
errupt
beahve
healthful
resepectful
shelves?no
reveiwing
descrimanting
appericiate
diragotori
cruptied
havve
tudor
unconvienient
gloryfiying
liberaters
idol's',and
spreading',yes
leasons
remane
know.but
shute
oergat
ales
porsn
rhit
bleva
sop
pelpar
reede
suet
bloe
bie
shaf
mexicans
gags
ossiated
unoppropiate
exspect
viwe
allown.because
it.same
fide
people.also
problem.i
do.heres
pizzy
conclusino
ringht
sceens
compaies
theings
gravitate
newsletter
relieze
unsuited
lives.well
alone.i
myslef
particcular
shouldn've
notbusy
relaxin
future's.schools
clases
invaded
wolds
didnt't
acreart
expecally
suppervised
condoms
drugs/@caps
hepls
fragrances
lotions
advive
youv'e
officave
book.there
bevlive
deminded
judgement.why
speeching
public.well
peoplle
freddom
autors
uterly
intsense
defference
selecion
u

hordes
tacky
oodles
world.censorship
rawness
interisting
book's
occiupied
bref
discripson
fravorite
meanning
lagacy
devestate
cencoroship
pooled
asume
unconstituitional
idealistic
countires
redemption
to.there
reffering
opposesed
argee
ergo
officals
synopsis
cookiee
volved
wactching
phiscally
tickect
disks
eseptible
aloun
caght
baned
mounths
cherrie
picker
liebrareis
-@num
buissy
cidren
upstars.they
capabull
duplicate
inlude
grouing
shoiuldent
emoutions
emoution
hatered
raceing
infinte
sustable
unholy
appauled
sceptical
boks
marred
exquisit
destroyer
takein
plite
keek
bestide
quesions
crated
offspring
principals
novelists
trembling
magnizes
imitating
responsiable
shiftless
inmportant
oursleves
deranged
secular
humanist
intergrated
inaliable
discreet
politic
deadline
impeachment
unintellilectual
announcing
coursebe
matures
cocasian
negroes
moives.so
restiction
zack
immorally
sheves
quiet.what
sinful
fine.that
ecept
tollerable
rabid
wolves
dosen'@caps
'?i
interresting
own.opinions
posiba

waas
evergreen
modernized
heatstore
peadiling
pargraph
directin
poarch
rosemite
rythm
psycoligical
include:-
caueses
malice
fountans
high
travilng
triying
toiledon
louage
jug
erodding
bes
bcouse
becasse
soid
life
californias
exguasted
.do
.these
.not
cyclistsoon
unoccupied
ridi
culously
shake
limits
land.
peddl
exauhtion
salvation
comesto
spectaculiar
chuckeled
fine,
deplets
wariness
larnest
psychologically
damaging,
roads
persimistic
salidrising
adaodoness
strote
adaodoned
dehydratiog
exausition
large
listned
decades.
suckenc
thirst
panice
exhusting
(pg
down.
nia
petal
frustating
ooted
envicus
envious
water
deplating
rheir
placewas
thirst,
turane
poys
desented
dangerousness
yuuth,
folluwed
ans
foctary
tostick
purchess
hce
excited
cicle
awee
tastd
polants
cockish
somelane
into?!
dehygrated
lijid
ohang
drippen
temperatare
clyclist
or,.
decades.
hasseled
strangers.
shortcut.when
fine.but
building.by
water.there
drink.he
easier.about
harder.towards
factory.thi

invirorment
happiiness
injoyes
differet
learded
lef
insinuates
revisits
sentence.he
ceartain
moood
old.@caps
describingwhat
oportinitys
imegrents
slefliss
durty
suprizing
soldarity
frendship
tempory
spesific
deatils
parints
exsepting
culenary
sacrificied
graditutity
mermoir
edication.the
confortible
home"this
soomthly
sentenee
prevaileel
neiborood
descring
culture.first
home.secondly
from.in
family"and
festivitys
fortunite
feelings.in
srprised
thanksful
better.all
becaause
bliuding
love.ii
thirs
fermiluar
enviting
comterable
apreshiates
nastest
cleanest
frienship
stying
leaes
sacrifize
detiration
neccisarily
furnshed
aparment
tackes
grateful.)"the
appreciat
decieds
haver
borned
enviroments
outgoing.the
friend.narciso
creatd
sacrifice.also
selfless.and
sacrifised
olwoys
greateful
obiously
him.this
person.in
parens
entirerly
jobs.the
becacus
badily
comsortabur
passioniate
senseful
"a
country.another
snippet
other.the
courage.in
sacraficid
withot
repeadtly
appreicative
memo
exaessed
eprss

ship.another
problems.to
enormousity
dangireous
itself.@caps
punture
weather.many
piolets
areas.the
buildings.it
developers
neiboring
cabled
weightless
viscinity
sucsesful
glimphs
dwarfed
saflley
informant
realzed
fam
securing
examles
struchure
prmitting
camo
firestarter.second
happen.third
collapein
visibly
elevated
nesescary
swivll
densey
safetly.another
tirering
repairs
inside-
dirigidle
catastrofick
flamming
plausibly
difficultys
annanced
there-
building.they
back.these
obstacles.the
dollars.consequently
wer'e
influentiel
diriglble
tethed
dangel
cited
impposible
resolt
saft
renevisions
idea.although
attahed
danger.the
balence
unforunantly
people.the
ancker
fruition
credibility
areas.first
fiasco.also
impractical.finally
law.in
winching
driigble
stacles
dockin
hypothetically
structuring
framework.lastly
citydwellers
incapability
strategically
flawed
unparallel
frow
cotastrophic
simpilest
whinch
bundles
citie
construction.if
projectoverall
spining
colapsing
factore
torde
acomidations

pere
shoews
magnum
the
intil
friendhips
chillout
deall
armadillos
smithing
snug
perparred
winey
yares
spriced
surnus
rubes
posable
patiets
wow!.
brothering
coro
hospitel
covent
nopthing
comimg
.tryouts
envelope,
.be
building.
.karma
bigs
rolercoster
statin
libaty
pikes
tarnado
laft
dropedme
jorney
yap
huston
wait.when
rocked
tthen
carolling
wistfull
bender
steair
tournament.
died.
filly
sooter
more
cprinted
darm
littlel
uptite
mackes
crancke
ceps
almost,
man!
sprinkling
weres
soey.
fine,
stired
done.
celery!
whinned
whn
dady
lowerd
awil
lightnig
cldnt
wouldhav
etrest
eanelusoin
beides
cuos
early?
happy
birthday
solos
streched
braclet
chipmunks
acorns
cookin
fusteration
comin
couisins
church
amen,
blubbered
said.@caps
bed.
ill
whaiting
aginizing
trowing
mandd
hours.
morning.
ughhh
accrcoss
cutine
diagnal
true.
ponytails
paintently
freax
flikering
line?
scaryed.
though.
ride?
with.
dozing
avon
orthodonist
expander
patience
tigers,
strait.
minite
hope

shush
pews
excercises
hymn
ridges
indented
hilariously
shafts
fogged
antiques
crevice
outliers
acrid
fermented
bras
echoes
barrels
mashed
mesh
zest
ketchup
kernels
pausing
reeking
drugstore
hangover
clinking
brunch
hoarse
thunderous
dab
jubilant
shutters
notorious
eying
loosely
remainder
monumented
retelling
cultivates
tepee
movie(i
sisters,@caps
macarana
springing
second.then
mash
medican"at
bummbed
siplest
beliving
apperant
shemakes
excample
stressfulness
laugh.and
seems(but
crunches
good.then
windiest
trolleys
chaperon
leasiurly
whatcha
goin'
councling
mediator
jokester
oddest
echoed
slouch
freecreditreport.com
whack
hamsters
downed
rafters
orford
lamas
lama
blistered
ignition
humored
partnerships
excursive
meadowed
overlooks
midmorning
tromping
shovels
chainsaws
trailbuilding
implements
exclaiming
tenants
birdhouses
treading
burbling
galumph
switchbacks
innertubes
pondweed
laura
turtles
darkening
hiccups
autumn
cider
mafia
nearsighted
indescribable
villainous
j
rosie
wasget
rampage

In [13]:
model = Word2Vec(allSentences, size=300, window=10, min_count=5, workers=3, iter=30)
model.save("Essayw2v.model")